<a href="https://colab.research.google.com/github/Jamess200/BirdnetProject/blob/main/Files/scripts/Drafts%26Preprocessing/Lost_Data_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Step 1: Imports**
# Import necessary libraries
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
#@title **Step 2: Clone the GitHub Repository**
# Clone the required GitHub repository into the Colab environment to access the data and scripts.

# Define the path where the repository will be cloned
repo_path = '/content/BirdnetProject'

# Check if the repository already exists
if not os.path.exists(repo_path):
    # Clone the repository if it doesn't exist
    !git clone https://github.com/Jamess200/BirdnetProject
else:
    # If the repository exists, pull the latest changes
    os.chdir(repo_path)
    !git pull

# Verify cloned repository
print("Repository contents:")
!ls /content/BirdnetProject

Already up to date.
Repository contents:
Files  README.md


In [ ]:
#@title **Step 3: Change Working Directory**
# Change the working directory to the path where the CSV data files are located.

# Change working directory to defined path
os.chdir('/content/BirdnetProject/Files/data/CSV_data')

# Verify working directory
print("Current working directory:", os.getcwd())

Current working directory: /content/BirdnetProject/Files/data/CSV_data


In [ ]:
#@title **Step 4: Data Loading and Preprocessing**
# Load the combined transect data, convert date and time columns to datetime format,
# and calculate detection intervals.

# Load the data
data = pd.read_csv('transect_combined.csv')

# Convert 'date' to datetime format
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

# Create 'datetime_start' and 'datetime_end' by combining 'date' with 'start_time' and 'end_time'
data['datetime_start'] = pd.to_datetime(data['date'].astype(str) + ' ' + data['start_time'], format='%Y-%m-%d %H:%M:%S')
data['datetime_end'] = pd.to_datetime(data['date'].astype(str) + ' ' + data['end_time'], format='%Y-%m-%d %H:%M:%S')

# Sort data by 'datetime_start'
data = data.sort_values(by='datetime_start')

# Calculate detection interval
data['detection_interval'] = data['datetime_start'].diff().shift(-1).dt.total_seconds()
average_interval = data['detection_interval'].mean()
print(f"Average detection interval: {average_interval:.2f} seconds")

Average detection interval: 13.07 seconds


In [ ]:
#@title **Step 5: Calculate Total Detections and Segments**
# Calculate the total number of detections, the expected number of detections, and the number of missing detections.

# Calculate total detections
total_detections = len(data)
print(f"Total number of detections: {total_detections}")

# Define recording schedule parameters
start_hour = 4
end_hour = 9
interval_minutes = 30
detection_interval = 3

# Calculate number of 3-second slots per 30-minute interval
slots_per_session = (interval_minutes * 60) / detection_interval

# Calculate number of sessions per day per transect
sessions_per_day_per_transect = 6

# Calculate number of slots per day per transect
slots_per_day_per_transect = slots_per_session * sessions_per_day_per_transect

# Define number of transects
number_of_transects = 4

# Calculate total expected slots per day across all transects
expected_slots_per_day = slots_per_day_per_transect * number_of_transects

# Calculate number of days in the dataset
data['day'] = data['date'].dt.date
total_days = data['day'].nunique()

# Calculate total number of expected detections
expected_detections = expected_slots_per_day * total_days
print(f"Expected number of detections: {expected_detections}")

# Calculate number of missing detections
missing_detections = expected_detections - total_detections
print(f"Number of missing detections: {missing_detections}")

Total number of detections: 54397
Expected number of detections: 129600.0
Number of missing detections: 75203.0


In [ ]:
#@title **Step 6: Detection Analysis by 3-Second Slots and Transects**
# Assign each detection to a 3-second slot and analyze the number of detections per slot and transect.

# Assign each detection to a 3-second slot
data['slot'] = data['datetime_start'].dt.floor('3S')

# Count the number of detections in each slot per transect
slot_counts_per_transect = data.groupby(['transect', 'slot']).size().reset_index(name='count')

# Identify slots with multiple detections within the same transect
multiple_detections_per_transect = slot_counts_per_transect[slot_counts_per_transect['count'] > 1]
print(f"Number of slots with multiple detections per transect: {multiple_detections_per_transect.shape[0]}")
print("Details of slots with multiple detections per transect:")
print(multiple_detections_per_transect)

# Extract hour from the 'slot' datetime and group by hour
multiple_detections_per_transect['hour'] = multiple_detections_per_transect['slot'].dt.hour
multiple_detections_per_hour_per_transect = multiple_detections_per_transect.groupby(['transect', 'hour']).size().reset_index(name='slots_with_multiple_detections')


Number of slots with multiple detections per transect: 3693
Details of slots with multiple detections per transect:
       transect                slot  count
10            5 2024-05-02 04:01:54      2
12            5 2024-05-02 04:02:06      2
32            5 2024-05-02 04:17:00      2
46            5 2024-05-02 04:18:39      2
195           5 2024-05-02 05:29:36      2
...         ...                 ...    ...
50561         8 2024-05-10 09:26:36      2
50564         8 2024-05-10 09:26:51      2
50569         8 2024-05-10 09:27:12      2
50573         8 2024-05-10 09:28:18      2
50584         8 2024-05-10 09:29:18      2

[3693 rows x 3 columns]


<ipython-input-17-9ca81bc6fcc8>:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#@title **Step 7: Visualize the Number of Multiple Detection Slots per Hour for Each Transect**
# Plot the number of multiple detection slots per hour for each transect.

fig = px.bar(
    multiple_detections_per_hour_per_transect,
    x='hour',
    y='slots_with_multiple_detections',
    color='transect',
    labels={'hour': 'Hour of Day', 'slots_with_multiple_detections': 'Number of Slots with Multiple Detections'},
    title='Number of Slots with Multiple Detections per Hour per Transect'
)

# Update layout to include legend and remove the color bar
fig.update_layout(
    legend_title_text='Transect',
    xaxis=dict(dtick=1),
    coloraxis_showscale=False,  # Remove the color bar
    showlegend=True  # Ensure the legend is shown
)

# Show the plot
fig.show()